<center>
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 2 - Código Base</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.00</h2>
</center>

# No debe utilizar bibliotecas adicionales.

In [ ]:
import numpy as np
import IPython
import time
import random
from scipy.io import wavfile
from scipy.signal import stft
from scipy.signal import istft
from scipy import linalg
from ipywidgets import interact
import pandas as pd
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

random.seed(0)

## Funciones&nbsp;entregadas:

In [ ]:
# Firma función 1
# Esta función de no debe ser modificada.
def WAV_to_Array(WAV_file, noise):
    """
    Parameters
    ----------
    WAV_file             : string 
                           WAV path
    noise                : boolean
                           flag for adding noise to sample

    Returns
    -------
    sample_rate          : int
                           Number of samples per second of the WAV file
    data                 : 1-D array                                       
                          NumPy array with WAV file data                   
    """                                                                    
    sample_rate, data = wavfile.read(WAV_file)
    if(len(data.shape) > 1): # En caso de que tenga más de un canal de audio, se trabaja con el primero
        data = data[:,0]
    data = data[:sample_rate*10]
    if noise:
        data = data + 0.01*np.asarray(random.sample(range(len(data)),len(data)))
    return sample_rate, data

# Firma función 2
# Esta función de no debe ser modificada.
def play_audio(data, sample_rate):
    """
    Parameters
    ----------
    data            : 1-D array                                       
                    NumPy array with WAV file data
    sample_rate     : int
                    Number of samples per second of the WAV file
    
    """
    IPython.display.display(IPython.display.Audio(data, rate=sample_rate))
    return

In [ ]:
# Ejemplo practico  cargada  y reproduccion de audio:
filename = "output/song_1/other.wav"
sample_rate_normal, data_normal = WAV_to_Array(filename, False)
sample_rate_noise, data_noise = WAV_to_Array(filename, True)
print("Audio normal:")
play_audio(data_normal , sample_rate_normal)
print("Audio con ruido:")
play_audio(data_noise , sample_rate_noise)

In [ ]:
# Esta función de no debe ser modificada.
def STFT(data):
    """
    Parameters
    ----------
    data                 : 1-D array                                      
                          NumPy array with WAV file data
    Returns
    -------
    M                : (p, q) array
                       M complex matrix
    """
    _, _, M = stft(data)
    return M
    
# Esta función de no debe ser modificada.
def ISTFT(M):
    """
    Parameters
    ----------
    M                : (p, q) array
                       M complex matrix
    Returns
    -------
    data             : 1-D array                                      
                        NumPy array with WAV file data
    """
    _, data = istft(M)
    return data

# Esta función de no debe ser modificada.
def Split(M):
    """
    Parameters
    ----------
    M                : (p, q) array
                       M complex matrix
    Returns
    -------
    M_Re             : (p, q) array
                       M_Re real matrix
    M_Im             : (p, q) array
                       M_Im real matrix
    """
    M_Re = M.real
    M_Im = M.imag
    return M_Re, M_Im

# Esta función de no debe ser modificada.
def Merge(M_Re, M_Im):
    """
    Parameters
    ----------
    M_Re             : (p, q) array
                       M_Re real matrix
    M_Im             : (p, q) array
                       M_Im real matrix
    Returns
    -------
    M                : (p, q) array
                       M complex matrix
    """
    M = M_Re + 1j*M_Im 
    return M

## Funciones&nbsp;a&nbsp;implementar:

In [ ]:
# M to PCA
def PCA_SVD(M, m):
    """
    Parameters
    ----------
    M             : (p, q) array
                    M real matrix
    m              : int
                    Number of components
    Returns
    -------
    V             : (q, m)-array
                     first m principal components
    Y              : (p,m)-array
                     Principal Component Coefficients
    mu             : (q)-array
                     Average per column 
    """
    # Apply PCA    
    mu = np.mean(M, axis = 0)
    
    #Centralization of variables
    Z = M - mu

    U,Sigma,V_con= np.linalg.svd(a=Z,full_matrices=False,compute_uv=True)
    
    #For real matrices V* is equivalent to V.T
    V = V_con.T[:,:m]
    
    #Sigma  is already sorted
    Sigma= np.diag(Sigma)
    
    # Y = U * Sigma
    Y = np.dot(U, Sigma)[:,:m]
    
    return V, Y, mu

# PCA to 'modified' M
def PCA_M(V, Y, mu):
    """
    Parameters
    ----------
    V              : (q, m)-array
                     first m principal components
    Y              : (p,m)-array
                     Principal Component Coefficients 
    mu             : (q)-array
                     Average per column 
    Returns
    -------
    Mm              : (p, q)-array
                     "Modified" M
    """
    #Reverse Process of PCA_SVD
    V_con=V.T
    Z = np.dot(Y,V_con)
    M = Z + mu
    return M

def calc_compression_ratio(data_length, M_shape, m1, m2):
    """
    Parameters
    ----------
    data_length          : length of 1-D array                                      
                          Length of the original NumPy array with WAV file data
    M_shape              : tuple
                          Complex M matrix Dimensions
    m1                   : int
                          Number of components to reduce M_Re real matrix
    m2                   : int
                          Number of components to reduce M_Im real matrix
    Returns
    -------
    compression_ratio  : Float
                         Compression ratio
    """
    
    # The size of the original is the number of the number of coefficients of the original Matrix
    original = data_length
    
    # The compressed size should be the sum of coeficients of the three matrices of the Real Matrix and the three matrices of the Imaginary Matrix
    
    _, c = M_shape
    real = c + m1 * np.sum(M_shape)
    imaginary = c + m2 * np.sum(M_shape)
    compressed = real + imaginary
    compression_ratio = (1 -(compressed / original)) * 100
    return compression_ratio



def compression_algorithm(data, m1, m2):
    """
    Parameters
    ----------
    data               : 1-D array                                      
                       NumPy array with WAV file data
    m1                 : int
                       Number of components to reduce M_Re real matrix
    m2                 : int
                       Number of components to reduce M_Im real matrix

    Returns
    -------
    V1             : (q, m)-array
                     first m principal components from M_Re
    Y1              : (p,m)-array
                     Principal Component Coefficients from M_Re
    mu1            : (q)-array
                     Average per column from M_Re
    V2             : (q, m)-array
                     first m principal components from M_Im
    Y2              : (p,m)-array
                     Principal Component Coefficients from M_Im
    mu2             : (q)-array
                     Average per column from M_Im
                
    """
    M = STFT(data)
    M_Re, M_Im = Split(M)
    V1, Y1, mu1 = PCA_SVD(M_Re, m1)
    V2, Y2, mu2 = PCA_SVD(M_Im, m2)
    return V1, Y1, mu1, V2, Y2, mu2

def decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2):
    """
    Parameters
    ----------
    V1             : (q, m)-array
                     first m principal components from M_Re
    Y1              : (p,m)-array
                     Principal Component Coefficients from M_Re
    mu1            : (q)-array
                     Average per column from M_Re
    V2             : (q, m)-array
                     first m principal components from M_Im
    Y2              : (p,m)-array
                     Principal Component Coefficients from M_Im
    mu2             : (q)-array
                     Average per column from M_Im

    Returns
    -------
    data_modified   : 1-D array                                      
                      NumPy array with WAV file data
    """
    M_Re = PCA_M(V1, Y1, mu1)
    M_Im = PCA_M(V2, Y2, mu2)
    M = Merge(M_Re, M_Im)
    data_modified = ISTFT(M)
    return data_modified



In [ ]:
matriz = np.random.randn(100, 100)
V, Y, mu = PCA_SVD(matriz, 1)
print(V)
print("-"*20)
print(Y)
print("-"*20)
print(mu)
print("-"*20)

## Experimentación

### El buen funcionamiento de la siguiente sección es un indicador de que su código funciona correctamente

In [ ]:
noise_reduction_attempts = [
    (7, 7),
    (8, 8),
    (9, 9),
    (10, 10),
    (20, 20),
    (35, 35),
    (45, 45),
    (55, 55),
    (65, 65),
    (75, 75),
    (85, 85),
    (95, 95),
    (100, 100)
]


def build_estimates(signal, noise_signal, n1_components, n2_components):
    M = STFT(signal)
    ratio = calc_compression_ratio(len(signal), M.shape, n1_components, n2_components)
    V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(noise_signal, n1_components, n2_components)
    signal_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)[:len(signal)]
    signal_Normalized = signal/linalg.norm(signal)
    signal_modified_Normalized = signal_modified/linalg.norm(signal_modified)
    error = np.linalg.norm(signal_Normalized - signal_modified_Normalized)
    mse = mean_squared_error(signal_Normalized, signal_modified_Normalized)
    return n1_components, ratio, error, mse, signal_modified

pca_compression_results = pd.DataFrame([
        build_estimates(data_normal, data_noise, n1, n2)
        for n1, n2 in noise_reduction_attempts
    ])
pca_compression_results.columns = ["Number of components", "Compression ratio", "error", "MSE", "Reconstructed signal"]
pca_compression_results

In [ ]:
print("Audio normal:")
play_audio(data_normal , sample_rate_normal)
print("Audio con ruido:")
play_audio(data_noise , sample_rate_noise)

In [ ]:
fig, ax = plt.subplots(figsize=(16,10),ncols=2, nrows=2)

## Graficar distintos tipos de errores
pca_compression_results.plot("Number of components", "error", "scatter", title='Error medio', ax=ax[0,0])
pca_compression_results.plot("Number of components", "MSE", "scatter", title='Error cuadrado medio', ax=ax[0,1])
pca_compression_results.plot("Number of components", "Compression ratio", "scatter", title='Análisis de Tasa de Compresión', ax=ax[1,0], label='PCA')
ax[1,0].plot([0,100], [0,0],'-', label='Sin PCA', color="orange")
# plt.plot(j, all_compressions_rates[j],'r.',ms=10)
ax[1,0].legend()
ax[0,0].grid(True)
ax[0,1].grid(True)
ax[1,0].grid(True)
plt.subplots_adjust(wspace=0.4, hspace=0.4)
plt.show()


In [ ]:
for reduced in pca_compression_results["Reconstructed signal"][:5]:
    play_audio(reduced , sample_rate_noise)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=65d1da02-617f-4545-8a92-006e4409c46c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>